In [31]:
import pandas as pd
from sparklines import sparklines
import numpy as np
import random
import glob
import csv, os
from matplotlib import pyplot as plt

import base64
import io


In [32]:
def spark_line(data, figsize=(3, 0.25), **kwags):
    """
    Returns a HTML image tag containing a base64 encoded sparkline style plot
    """
    data = list(data)

    fig, ax = plt.subplots(1, 1, figsize=figsize, **kwags)
    ax.plot(data)
    for k,v in ax.spines.items():
        v.set_visible(False)
    ax.set_xticks([])
    ax.set_yticks([])

    plt.plot(len(data) - 1, data[len(data) - 1], 'r.')

    ax.fill_between(range(len(data)), data, len(data)*[min(data)], alpha=0.1)

    img = io.BytesIO()
    plt.savefig(img, transparent=True, bbox_inches='tight')
    img.seek(0)
    plt.close()

    return base64.b64encode(img.read()).decode("UTF-8")

In [33]:
def is_file_empty(file_path):
    return os.path.exists(file_path) and os.stat(file_path).st_size == 0

In [34]:
df1 = list()
for files in glob.glob('.\\Databases\\*.csv'):
    f=open(files, 'r')
    name = os.path.basename(f.name).replace(".csv","")
    if is_file_empty(".\\Databases\\"+name+".csv"):
        continue
    sensex = pd.read_csv(f)
    try:
        df1.append(sensex["Close"][0:30].values)
    except Exception as e:
        df1.append(sensex["Close Price"][0:30].values)

with open("bse_closing_price.html", "a") as file:
    file.write('<div><h1>Sparkline to the Closing Price values</h1></div>')
    file.write('<br>')
    file.write('<br>')
    for value in df1:
        file.write('<div><img src="data:image/png;base64,{}"/></div>'.format(spark_line(value)))
        file.write('<div></div>')

In [35]:
import pdfcrowd

In [36]:
client = pdfcrowd.HtmlToPdfClient('Madhuri97', '2b316e93793442d8031a72d908db069c')
client.convertFileToFile('bse_closing_price.html', 'bse_closing_price.pdf')